<a href="https://www.kaggle.com/ismailaseck/spp-mlp?scriptVersionId=86883422" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import SGD
import torch.nn.functional as F 

from sklearn.impute import KNNImputer # to deal with missing values

from sklearn.preprocessing import MinMaxScaler # to rescale the data

from seaborn import histplot

# loading the data

and get basic information about it

In [ ]:
# loading the data
train_original = pd.read_csv("/kaggle/input/song-popularity-prediction/train.csv")
test_original = pd.read_csv("/kaggle/input/song-popularity-prediction/test.csv")
sample_submission = pd.read_csv("/kaggle/input/song-popularity-prediction/sample_submission.csv")

In [ ]:
train_original.info()

In [ ]:
for c in train_original.columns:
    c_unique =  np.unique(train_original[c].dropna()) # dropping the nan then counting the number of unique values
    print(c, len(c_unique))

So key, audio_mode and time_signature are categorical features. We are going to use embeddings to deal with them and harness the power of multi-layer perceptron. We inform the train_original dataframe that those features are categorical in the following cell. 

In [ ]:
to_categorical = {'key': 'category', 'audio_mode':'category', 'time_signature':'category'}

for k in to_categorical:
    train_original[k].astype('category')
    test_original[k].astype('category')
#train_original.info()


In [ ]:
train_original.columns

# dealing with missing values using the KNNImputer of sklearn

1. We will first rescale all the numerical columns to be between -0.5 and 0.5 to have the same order of magnitude when using the KNNImputer.

2. Then we use the KNNImputer to fit the numerical columns using the mean of the nearest neighbors.

3. Then we use the KNNImputer to fit the categorical values using the most represented category. 

In [ ]:
# rescaling
rescaled_train = train_original.copy()
rescaled_test = test_original.copy()
ignore_columns = ['id', 'audio_mode', 'time_signature','song_popularity']
scaler_dict = {}

for column in train_original.columns:
    if column not in ignore_columns:
        print(column)
        scaler = MinMaxScaler(feature_range=(-0.5,0.5)) 
        transformed_column = scaler.fit_transform(train_original[column].values.reshape(40000,1))
        rescaled_train[column] = transformed_column.squeeze()
        rescaled_test[column] = scaler.transform(test_original[column].values.reshape(10000,1))
        scaler_dict[column] = scaler
        


In [ ]:
scaler_dict

In [ ]:
rescaled_train

In [ ]:
train_original

# comment : 
The only categorical feature having missing values is 'key'. And since 'audio_mode' takes only two values, 0 and 1, it does not matter if it is ordinal or not.
Regarding the 'time_signature', we assume it is ordinal and hence can be used to compute distance. Furthermore, 'time_signature' takes only 4 values [2,3,4,5] and mostly takes value 3 and 4 and rarely 2 and 5.
So we are going to consider them as ordinale and use them to compute distances in the KNNImputer. 

In [ ]:
#unique, count = np.unique(train_original["time_signature"], return_counts=True)
#unique, count # change to histogram when sharing 
histplot(train_original["time_signature"], discrete=True)


### Imputing continuous values

In [ ]:

imputer_continuous = KNNImputer(n_neighbors=15,weights='distance')
imputed_train = rescaled_train.copy()
imputed_test = rescaled_test.copy()
# let the imputation use the song_popularity column and add song_popularity to the test data but
# set it to 0.5 so that it does not contribute to the imputation. 
imputed_train = imputed_train.drop(columns='id') 
imputed_test = imputed_test.drop(columns='id')
imputed_test['song_popularity']= 0.5

imputed_train = pd.DataFrame(imputer_continuous.fit_transform(imputed_train), columns = imputed_train.columns)
imputed_test = pd.DataFrame(imputer_continuous.transform(imputed_test), columns = imputed_test.columns)



### Imputing the categorical value, 'key'

In [ ]:
imputer_key = KNNImputer(n_neighbors=1)
rescaled_train['key'] = train_original['key']
rescaled_test['key'] = test_original['key']

#print(imputed_train.info())
temp = pd.DataFrame(imputer_key.fit_transform(rescaled_train.drop(columns='id')), columns = imputed_train.columns)
imputed_train['key'] = temp['key']
imputed_train

rescaled_test['song_popularity']=0.5
temp = pd.DataFrame(imputer_key.transform(rescaled_test.drop(columns='id')), columns = imputed_train.columns)
imputed_test['key'] = temp['key']
imputed_test = imputed_test.drop(columns='song_popularity') # drop the song popularity column since we do not need it anymore
imputed_test.info()

In [ ]:
train_original

In [ ]:
imputed_test

### Preparing the dataset to be used with dataloader

In [ ]:
new_order = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'audio_valence','key', 'audio_mode', 'time_signature',
       ]
reord_imputed_train=imputed_train[new_order]
reord_imputed_train_label = imputed_train['song_popularity']
reord_imputed_test = imputed_test[new_order]

In [ ]:
x_train = torch.from_numpy(reord_imputed_train.values)
y_train = torch.from_numpy(reord_imputed_train_label.values)
x_test = torch.from_numpy(reord_imputed_test.values)
train_dataset = TensorDataset(x_train,y_train)
train_loader = DataLoader(train_dataset, batch_size=256,shuffle=True)
test_dataset = TensorDataset(x_test)
test_loader = DataLoader(test_dataset, batch_size=128,shuffle=False)

# creating the spp_mlp class, a neural network class to predict 
Hopefully predictions are more accurate than tree based models' prediction.  

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
class spp_mlp(nn.Module):
    def __init__(self, num_neurons=64,drop_prob=0.00, lrelu_slope=0.00):
        super(spp_mlp, self).__init__()
        dim_continuous = 10
        dim_key_embedding = 4
        dim_ts_embedding = 2
        dim_am_embedding = 2
        dim_mlp_input = dim_continuous + dim_key_embedding + dim_ts_embedding + dim_am_embedding
        self.key_embedding = nn.Embedding(12,dim_key_embedding).to(device) # 12 different values and each being embedded in a dim_key_embedding-dimensional vector
        self.time_signature_embedding= nn.Embedding(4,dim_ts_embedding).to(device)
        self.audio_mode_embedding = nn.Embedding(2,dim_am_embedding).to(device)
        self.fc1 = nn.Linear(dim_mlp_input,num_neurons)
        self.fc2 = nn.Linear(num_neurons,num_neurons)
        self.output = nn.Linear(num_neurons,2)
        self.dropout = nn.Dropout(p=drop_prob)
        self.lrelu = nn.LeakyReLU(lrelu_slope)
    
    def forward(self,x):
        continuous = x[:,:10].to(device)
        categorical = x[:,10:].type(torch.LongTensor).to(device)
        key_embedding = self.key_embedding(categorical[:,0]).clone().to(device)
        am_embedding =self.audio_mode_embedding(categorical[:,1]).clone().to(device)
        ts_embedding = self.time_signature_embedding(categorical[:,2]-2).clone().to(device) # -2 to bring the values from [2,3,4,5] to [0,1,2,3]
        
        
        tensors_list = (continuous,key_embedding,ts_embedding,am_embedding)
        mlp_input = torch.cat(tensors_list,1).to(device)
        #print(mlp_input.shape)
        
        fc1 = self.fc1(mlp_input)
        fc1 = self.lrelu(fc1)
        fc1 = self.dropout(fc1)
        
        fc2 = self.fc2(fc1)
        fc2 = self.lrelu(fc2)
        fc2 = self.dropout(fc2)
        
        logit =self.output(fc2)
        
        return logit
    
    def train_1epoch(self,train_loader, optimizer, device = 'cpu'):
        self.train()
        weight = torch.tensor([1./3,2./3]).double()
        for batch_x, batch_y in train_loader:
            logit = self(batch_x.to(device))
            label = batch_y.type(torch.LongTensor).to(device)
            
            
            #loss = F.cross_entropy(logit,label,weight)
            loss = F.cross_entropy(logit.to(device),label.to(device))
            #print(f'loss before backward: {loss}')
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)#https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html
            
            #logit = self(batch_x)
            #loss = F.cross_entropy(logit,label)
            #print(f'loss after backward {loss}')
            
            
    def evaluate(self,data_loader,device='cpu'):
        self.eval()
        eval_loss =0
        accuracy = 0
        w = torch.tensor([1./3,2./3]).double()
        
        #with torch.no_grad():
        for batch_x, batch_y in data_loader:
            logit = self(batch_x.to(device))
            label = batch_y.type(torch.LongTensor).to(device)
            #eval_loss += F.cross_entropy(logit, label, w,  reduction='sum')
            eval_loss += F.cross_entropy(logit.to(device), label.to(device),  reduction='sum')
            pred = logit.argmax(dim=int(1)).to(device)
            accuracy += torch.sum(pred==label).double()
        n_elements = len(data_loader.dataset)
        return accuracy/n_elements, eval_loss/n_elements

### training the model 




In [ ]:
from torch.optim import AdamW

mlp = spp_mlp(num_neurons=1024)
mlp.double()
mlp = mlp.to(device)


accuracy, loss = mlp.evaluate(train_loader)
print(f'the accuracy before trainig is : {accuracy} and the average loss is : {loss}')

num_epoch = 10
optimizer = AdamW(mlp.parameters(), lr = 5e-5)
for i in range(num_epoch):
    mlp.train_1epoch(train_loader, optimizer)
    accuracy, loss = mlp.evaluate(train_loader)
    print(f'epoch : {i} accuracy: {accuracy:.4f} loss: {loss:.4f}')
    

In [ ]:
for i in range(10):
    mlp.train_1epoch(train_loader, optimizer)
    accuracy, loss = mlp.evaluate(train_loader)
    #if i%4==0:
    print(f'epoch : {i} accuracy: {accuracy:.4f} loss: {loss:.4f}')

In [ ]:
# compute AUC on the training set 
from sklearn.metrics import roc_auc_score

logit = mlp(train_dataset[:][0])
softmax = F.softmax(logit,dim=1).cpu().detach().numpy()
label = train_dataset[:][1]
train_auc = roc_auc_score(label,softmax[:,1])
print(train_auc)

In [ ]:
torch.save(mlp.state_dict(), "/kaggle/working/spp_mlp_embedding.pth")

In [ ]:
#optimizer.param_groups[0]['lr'] *= 0.75
 #print(optimizer.param_groups[0]['lr'])

In [ ]:
logit = mlp(test_dataset[:][0])
softmax = F.softmax(logit).cpu().detach().numpy()
softmax, logit

In [ ]:
# submission 
submission = pd.read_csv("../input/song-popularity-prediction/sample_submission.csv")


logit = mlp(test_dataset[:][0])
softmax = F.softmax(logit).cpu().detach().numpy()
#label = test_dataset[:][1]

submission['song_popularity'] = softmax[:,1]
submission
#submission.to_csv(f"sample_submission_{train_auc:.2f}_on_train.csv", index=False)
submission.to_csv("sample_submission.csv",index=False)